In [1]:
import torch
from datasets import IndexedDataset, WeightedDataset
from torch.utils.data import DataLoader, DistributedSampler

from utils import get_args, get_exp_name
from architectures import load_architecture

from samplers import InfoBatch , CustomSampler
from losses import get_loss
from tqdm.notebook import tqdm

args = get_args()
# args.arch = 'LeNet5'
# args.dataset = 'MNIST'
# args.selection_method = 'none'

# Example Usage

# model, target_layers = load_architecture(args)

# get_exp_name(args)

# args.iterations = 2
# args.ratio = 0.5
# args.delta = 1
# args.batch_size = 128
# args.pruner = 'score'
# args.sample_size=256

# # train_dataset = IndexedDataset()
# print('init weighted dataset')
# train_dataset = WeightedDataset(args, epochs = args.epochs, train=True, prune_ratio = args.ratio, delta = args.delta, )

# print('init sampler')
# sampler = CustomSampler(args, train_dataset)

# # trainset = InfoBatch(dataset, args.epochs, args.ratio, args.delta)

# # # sampler = trainset.sampler
# # train_shuffle = False

# print('init dataloder')
# # trainloader = DataLoader(train_dataset, batch_size=None, num_workers=0, sampler=sampler) # 
# trainloader = DataLoader(train_dataset, batch_size=None, num_workers=0, sampler = sampler,  pin_memory=True) # 

# print('init optimizer')
# optimizer = torch.optim.Adam( model.parameters(), lr=args.lr, )

# def train_info_batch(epoch):
#     # safe_print('\nEpoch: %d, iterations %d' % (epoch, len(trainloader)))
#     model.train()

#     for batch_idx, blobs in  tqdm( enumerate(trainloader) ) :
#         # print(len(blobs) )
#         inputs, targets, idxs = blobs 
#         # print(idxs)
#         # print(targets)

#         # print(len(batch))
#         inputs, targets = inputs.to('cuda'), targets.to('cuda')

#         # train_dataset.set_active_indices(idxs)
#         optimizer.zero_grad()

#         loss_values, clean_values, robust_values, logits_nat, logits_adv = trades_loss(model=model, x_natural=inputs, y=targets, optimizer=optimizer,)
#         # 3. use <InfoBatch>.update(loss), all scoring/rescaling/getting mean is now conducted at the backend, see previous (research version) code for details.

#         train_dataset.update_scores(idxs, clean_values, robust_values, loss_values, logits_nat, logits_adv)
#         # print(trainset.weights.shape)
#         # print(loss_values.shape)
#         # print(trainset.cur_batch_index)
#         loss = train_dataset.compute_weighted_loss(idxs, loss_values)

#         # print(loss)
#         loss.backward()
#         # print(loss)
        
#     # safe_print('epoch:', epoch, '  Training Accuracy:', round(100. * correct /
#     #     total, 3), '  Train loss:', round(train_loss / len(trainloader), 4))
#     # train_acc.append(correct / total)

# for epoch in range(args.epochs):
#     print('epoch')
#     # trainloader.sampler.set_epoch(epoch)
#     train_info_batch(epoch) 

./data


In [1]:
import torch
from datasets import IndexedDataset, WeightedDataset
from torch.utils.data import DataLoader, DistributedSampler

from utils import get_args
from architectures import load_architecture

from samplers import DistributedCustomSampler
from losses import trades_loss
from tqdm.notebook import tqdm

args = get_args()
args.arch = 'LeNet5'
args.dataset = 'random'
args.selection_method = 'none'

# Example Usage

model, target_layers = load_architecture(args)

args.epochs = 2
args.pruning_ratio = 0
args.delta = 1
args.batch_size = 2
args.pruning_strategy = 'random'
args.batch_strategy = 'random'
args.sample_size= 5

# train_dataset = IndexedDataset()
print('init weighted dataset')
train_dataset = WeightedDataset(args, train=True, prune_ratio = args.pruning_ratio,  )

# for i in range(len(train_dataset)):
#     print(train_dataset[i])

dist_sampler = DistributedCustomSampler(args, train_dataset, num_replicas=2, rank=1, drop_last=True)

print('init dataloder')
trainloader = DataLoader(train_dataset, batch_size=None, sampler = dist_sampler,)  


keep_track = []

for iteration in range(100):
    
    print()
    print('Start of iteration', iteration)
    print()
    
    dist_sampler.set_epoch(iteration)
    
    for batch_id, batch in enumerate( trainloader ):
        print('batch no.', batch_id,batch)
        data, label, idx = batch
        keep_track.append( data[0].tolist()[0] )




./data
init weighted dataset
init dataloder

Start of iteration 0

global [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
pruning
post_pruning [2, 8, 4, 9, 1, 6, 7, 3, 0, 5]
remove tail
process [8, 9, 6, 3, 5]
batch no. 0 [tensor([[8],
        [9]]), tensor([8, 9]), tensor([8, 9])]
batch no. 1 [tensor([[6],
        [3]]), tensor([6, 3]), tensor([6, 3])]
batch no. 2 [tensor([[5]]), tensor([5]), tensor([5])]

Start of iteration 1

global [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
pruning
post_pruning [2, 9, 6, 4, 0, 3, 1, 7, 8, 5]
remove tail
process [9, 4, 3, 7, 5]
batch no. 0 [tensor([[9],
        [4]]), tensor([9, 4]), tensor([9, 4])]
batch no. 1 [tensor([[3],
        [7]]), tensor([3, 7]), tensor([3, 7])]
batch no. 2 [tensor([[5]]), tensor([5]), tensor([5])]

Start of iteration 2

global [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
pruning
post_pruning [4, 1, 5, 0, 7, 2, 3, 6, 9, 8]
remove tail
process [1, 0, 2, 6, 8]
batch no. 0 [tensor([[1],
        [0]]), tensor([1, 0]), tensor([1, 0])]
batch no. 1 [tensor([[2],
        [6

In [4]:
set(keep_track)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}